# CODE TO WORK THROUGH CUSTOM IMPLEMENTATION. 

## Idea is to raise error at every collision, implement custom code to extract velocities from the planetesimal involved in collision at time of collision, and store the value. 

Define functions that will be used in main code block (specifically, these functions are used to assign orbital 
parameters for the planetesimals)

In [186]:
def rand_powerlaw(slope, min_v, max_v):
    y = np.random.uniform()
    pow_max = pow(max_v, slope+1.)
    pow_min = pow(min_v, slope+1.)
    return pow((pow_max-pow_min)*y + pow_min, 1./(slope+1.))

def rand_uniform(minimum, maximum):
    return np.random.uniform()*(maximum-minimum)+minimum

def rand_rayleigh(sigma):
    return sigma*np.sqrt(-2*np.log(np.random.uniform()))

Define function that streamlines simulation set-up using: 

'direct' collision detection /
'mercurius' integrator /
no built-in collision resolver (trying to build custom resolve) 

In [187]:
def setupSimulation():
    
    sim = rebound.Simulation()
    sim.integrator = "mercurius"
    sim.dt = 0.000206968604*2*np.pi 
    sim.testparticle_type = 1 # set planetesimals to semi-active mode
    sim.ri_ias15.min_dt = 1e-6 # ensure that close encounters do not stall the integration. This sets a minimal time step 
                           # for IAS15. 
    
    sim.collision = "direct"
    #sim.collision_resolve = "merge" # merge any planetesimals that collide with a planet, conserving momentum and mass

    sim.boundary = "open" # open boundary conditions. Remove particle if they leave the box (are ejected from system)
    #boxsize = 0.0100003232
    boxsize = 2
    sim.configure_box(boxsize)

    # sim.track_energy_offset = 1 # track the energy lost due to ejections and collisions
    
    mEarth = 3e-6 # Earth mass in units of Msun
    rEarth = 4.25e-5 # Earth radius in units of AU 

    ## all values dervied from Agol et al. True anomalies assuming perfectly circular orbits 
    sim.add(m=.0898, hash="star")
    sim.add(m=1.374*mEarth, r=1.116*rEarth, a=0.01154, e=0.00305473, omega=2.35156489, f=1.62005, inc=0.004747296, 
           hash="b") # Trappist1b
    sim.add(m=1.308*mEarth, r=1.097*rEarth, a=0.01580, e=0.00055009, omega=0.01817982, f=4.448437, inc=0.003874631,
           hash="c") # Trappist1c
    sim.add(m=0.388*mEarth, r=.788*rEarth, a=0.02227, e=0.00563298, omega=2.64777152, f=5.293947, inc=0.001815142,
           hash="d") #Trappist1d
    sim.add(m=0.692*mEarth, r=.920*rEarth, a=0.02925, e=0.00632463, omega=0.81670784, f=2.183237, inc=0.003612832,
           hash="e") #Trappist1e
    sim.add(m=1.039*mEarth, r=1.045*rEarth, a=0.03849, e=0.00841547, omega=0.06063985, f=5.09579, inc=0.004537856,
           hash="f") #Trappist1f
    sim.add(m=1.321*mEarth, r=1.129*rEarth, a=0.04683, e=0.00400979, omega=0.32490512, f=4.12414, inc=0.004502949,
           hash="g") #Trappist1g
    sim.add(m=0.326*mEarth, r=0.755*rEarth, a=0.06189, e=0.00365005, omega=0.0054794, f= 4.7124, inc=0.003403392,
           hash="h") #Trappist1h

    mTrappistH=0.326*mEarth
    aTrappistH = 0.06189

    mSun=.0898

    hillRadiusH = aTrappistH * (mTrappistH / mSun) ** (1/3)

    sim.N_active = 8 # declares amount of active bodies in the system. All other bodies added after are semi-active
    
    N_pl = 5 # number of planetesimals 
    m_pl = mEarth*(1e-5) # mass of each planetesimal
    r_pl = 1.00446e-6 # radius of each planetesimal 

    np.random.seed(42) # By setting a seed we will reproduce the same simulation every time (easier for debugging). 
                   # Remove seed if you want randomized simulation runs. 
    while sim.N < (N_pl + sim.N_active):
        a = rand_powerlaw(0, aTrappistH + hillRadiusH, aTrappistH + hillRadiusH + .005)  
        e = rand_rayleigh(0.04) # Original value .01
        inc = rand_rayleigh(0.005)
        f = rand_uniform(-np.pi,np.pi)
        p = rebound.Particle(simulation=sim,primary=sim.particles[0],m=m_pl, r=r_pl, a=a, e=e, inc=inc, Omega=0, omega=0, f=f)
        # Only add planetesimal if it's far away from the planet
        d = np.linalg.norm(np.array(p.xyz)-np.array(sim.particles[1].xyz))
        if d>0.01:
            sim.add(p)
            
    sim.move_to_com() # Move system to Center of Mass reference frame
    
    return sim

# This code block uses built in merge function. 

## Use to check against results of custom resolution. 

### End goal: custom collision resolution code should result in identical simulation status as built-in resolution except with the ability to extract additional data. 

In [188]:
%%time
    
import rebound
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from rebound import hash as h
import pandas as pd
%matplotlib inline

sim = rebound.Simulation()
sim.integrator = "mercurius"
sim.dt = 0.000206968604*2*np.pi 
sim.testparticle_type = 1 # set planetesimals to semi-active mode
sim.ri_ias15.min_dt = 1e-6 # ensure that close encounters do not stall the integration. This sets a minimal time step 
                           # for IAS15. 
    
sim.collision = "direct"
sim.collision_resolve = "merge" # merge any planetesimals that collide with a planet, conserving momentum and mass

sim.boundary = "open" # open boundary conditions. Remove particle if they leave the box (are ejected from system)
#boxsize = 0.0100003232
boxsize = 2
sim.configure_box(boxsize)
    
mEarth = 3e-6 # Earth mass in units of Msun
rEarth = 4.25e-5 # Earth radius in units of AU 

## all values dervied from Agol et al. True anomalies assuming perfectly circular orbits 
sim.add(m=.0898, hash="star")
sim.add(m=1.374*mEarth, r=1.116*rEarth, a=0.01154, e=0.00305473, omega=2.35156489, f=1.62005, inc=0.004747296, 
        hash="b") # Trappist1b
sim.add(m=1.308*mEarth, r=1.097*rEarth, a=0.01580, e=0.00055009, omega=0.01817982, f=4.448437, inc=0.003874631,
        hash="c") # Trappist1c
sim.add(m=0.388*mEarth, r=.788*rEarth, a=0.02227, e=0.00563298, omega=2.64777152, f=5.293947, inc=0.001815142,
        hash="d") #Trappist1d
sim.add(m=0.692*mEarth, r=.920*rEarth, a=0.02925, e=0.00632463, omega=0.81670784, f=2.183237, inc=0.003612832,
        hash="e") #Trappist1e
sim.add(m=1.039*mEarth, r=1.045*rEarth, a=0.03849, e=0.00841547, omega=0.06063985, f=5.09579, inc=0.004537856,
        hash="f") #Trappist1f
sim.add(m=1.321*mEarth, r=1.129*rEarth, a=0.04683, e=0.00400979, omega=0.32490512, f=4.12414, inc=0.004502949,
        hash="g") #Trappist1g
sim.add(m=0.326*mEarth, r=0.755*rEarth, a=0.06189, e=0.00365005, omega=0.0054794, f= 4.7124, inc=0.003403392,
        hash="h") #Trappist1h

mTrappistH=0.326*mEarth
aTrappistH = 0.06189

mSun=.0898

hillRadiusH = aTrappistH * (mTrappistH / mSun) ** (1/3)

sim.N_active = 8 # declares amount of active bodies in the system. All other bodies added after are semi-active
    
N_pl = 5 # number of planetesimals 
m_pl = mEarth*(1e-5) # mass of each planetesimal
r_pl = 1.00446e-6 # radius of each planetesimal 

np.random.seed(42) # By setting a seed we will reproduce the same simulation every time (easier for debugging). 
                   # Remove seed if you want randomized simulation runs. 
while sim.N < (N_pl + sim.N_active):
    a = rand_powerlaw(0, aTrappistH + hillRadiusH, aTrappistH + hillRadiusH + .005)  
    e = rand_rayleigh(0.04) # Original value .01
    inc = rand_rayleigh(0.005)
    f = rand_uniform(-np.pi,np.pi)
    p = rebound.Particle(simulation=sim,primary=sim.particles[0],m=m_pl, r=r_pl, a=a, e=e, inc=inc, Omega=0, omega=0, f=f)
    # Only add planetesimal if it's far away from the planet
    d = np.linalg.norm(np.array(p.xyz)-np.array(sim.particles[1].xyz))
    if d>0.01:
        sim.add(p)
            
sim.move_to_com() # Move system to Center of Mass reference frame

# print(sim.status()) 

s = (sim.particles[h("star")])
b = (sim.particles[h("b")])
c = (sim.particles[h("c")])
d = (sim.particles[h("d")])
e = (sim.particles[h("e")])
f = (sim.particles[h("f")])
g = (sim.particles[h("g")])
h = (sim.particles[h("h")])

endTime = 2.6834184*2*np.pi
#endTime = 2.6835*2*np.pi

sim.integrate(endTime)

"""
timesteps = 100

# Create array that will store orbital information for all bodies
# orbitalInformation = np.zeros((timesteps, 7)) # First value = number of timesteps, Third value = # of orbital parameters
times = np.linspace(0.,endTime,timesteps) # Create timesteps

bAU = []; cAU = []; dAU = []; eAU = []; fAU = []; gAU = []; hAU = [];

for i,t in enumerate(times):
        
    sim.integrate(t,exact_finish_time=1)
    
    bAU.append(b.a); cAU.append(c.a); dAU.append(d.a); eAU.append(e.a); fAU.append(f.a); gAU.append(g.a); hAU.append(h.a)
    
orbitalInformation = pd.DataFrame({'Time':times, 'b AU':bAU, 'c AU':cAU, 'd AU':dAU, 'e AU':eAU, 'f AU':fAU, 
                                   'g AU':gAU, 'h AU':hAU})  
print(orbitalInformation)
"""
rBuilt = h.r
mBuilt = h.m
vxBuilt = h.vx
vyBuilt = h.vy
vzBuilt = h.vz
iBuilt = h.inc
eBuilt = h.e
xBuilt = h.x
yBuilt = h.y
zBuilt = h.z
print(sim.status())
comBuilt = sim.calculate_com()

---------------------------------
REBOUND version:     	3.17.0
REBOUND built on:    	May  6 2021 14:38:50
Number of particles: 	12
Selected integrator: 	mercurius
Simulation time:     	1.6860415063895353e+01
Current timestep:    	0.001300
---------------------------------
<rebound.particle.Particle object at 0x1214c1ec0, m=0.0898 x=-9.36351152343433e-07 y=-2.1473566937323624e-06 z=-8.858633407927266e-09 vx=0.00016078782346911046 vy=9.711762376996173e-05 vz=4.624551484336938e-07>
<rebound.particle.Particle object at 0x12157b940, m=4.1220000000000005e-06 x=-0.0050675045808504385 y=0.01032930327563014 z=4.747224842380202e-05 vx=-2.509194097771507 vy=-1.2364300101652588 vz=-0.006382892402831585>
<rebound.particle.Particle object at 0x1214c1ec0, m=3.924e-06 x=-0.012554286344502102 y=-0.009630835642532674 z=-3.630103450267814e-05 vx=1.4503202362237844 vy=-1.8883963111873314 vz=-0.007520806071238305>
<rebound.particle.Particle object at 0x12157b940, m=1.164e-06 x=0.006970175847473721 y=0.0210

# TRYING TO DEBUG THIS CUSTOM RESOLUTION MODEL. 
# Custom resolution conserving mass, volume, momentum. 

## Following same structure as Rebound github > src > collisions.c

### Outputs a dataframe for each collision with time, impacted body, relative velocity, and total impacts for that body

In [189]:
%%time

# import required packages
import rebound
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from rebound import hash as h
import pandas as pd

%matplotlib inline

sim = setupSimulation()

endTime = 2.6834184*2*np.pi

starImpacts = 0; bImpacts = 0; cImpacts = 0; dImpacts = 0; eImpacts = 0; fImpacts = 0; gImpacts = 0; hImpacts = 0

star = (sim.particles[h("star")]); b = (sim.particles[h("b")]); c = (sim.particles[h("c")]); d = (sim.particles[h("d")]); 
e = (sim.particles[h("e")]); f = (sim.particles[h("f")]); g = (sim.particles[h("g")]); h = (sim.particles[h("h")])

# initialize columns for dataframe
impactedBodies = []; impactTimes = []; impactVelocity = []; totalImpacts = [] 

while (sim.t != endTime): # while you still haven't reached the end of the intended simulation time... 
    
    try: # try to integrate to end of simulation time... 
        
        sim.integrate(endTime)
    
    except: # except if you run into an error (collision), run the following code: 
        
        collided = [] # initialize array that will store indices of collided particles 
        
        collisionTimes = []
        
        for particle in sim.particles: # loop through all particles in the simulation...  
            
            collisionTimes.append(particle.lastcollision)
            
        latestCollision = max(collisionTimes)
        
        for i in xrange(len(collisionTimes)):
            
            if collisionTimes[i] == latestCollision: # find the particles involved in the collision... 
           
                p = sim.particles[i]
                    
                collided.append(p.index) # add particle index to collided array
                
                # Hardcoding to find the planet involved in collision by comparing particle parameters with 
                # planet parameters (retrieved by the hashes we declared in simulation set-up)
                if p.x == star.x and p.y == star.y and p.z == star.z and \
                    p.vx == star.vx and p.vy == star.vy and p.vz == star.vz:                                                 
                    impactedBody = 'star'
                    starImpacts += 1  
                    impactTracker = starImpacts
                    planetVelocityComp = [p.vx, p.vy, p.vz]
                elif p.x == b.x and p.y == b.y and p.z == b.z and p.vx == b.vx and p.vy == b.vy and p.vz == b.vz: 
                    impactedBody = 'b'
                    bImpacts += 1
                    impactTracker = bImpacts
                    planetVelocityComp = [p.vx, p.vy, p.vz]
                elif p.x == c.x and p.y == c.y and p.z == c.z and p.vx == c.vx and p.vy == c.vy and p.vz == c.vz: 
                    impactedBody = 'c'
                    cImpacts += 1
                    impactTracker = cImpacts
                    planetVelocityComp = [p.vx, p.vy, p.vz]
                elif p.x == d.x and p.y == d.y and p.z == d.z and p.vx == d.vx and p.vy == d.vy and p.vz == d.vz: 
                    impactedBody = 'd'
                    dImpacts += 1
                    impactTracker = dImpacts
                    planetVelocityComp = [p.vx, p.vy, p.vz]
                elif p.x == e.x and p.y == e.y and p.z == e.z and p.vx == e.vx and p.vy == e.vy and p.vz == e.vz: 
                    impactedBody = 'e'
                    eImpacts += 1
                    impactTracker = eImpacts
                    planetVelocityComp = [p.vx, p.vy, p.vz]
                elif p.x == f.x and p.y == f.y and p.z == f.z and p.vx == f.vx and p.vy == f.vy and p.vz == f.vz: 
                    impactedBody = 'f'
                    fImpacts += 1
                    impactTracker = fImpacts
                    planetVelocityComp = [p.vx, p.vy, p.vz]
                elif p.x == g.x and p.y == g.y and p.z == g.z and p.vx == g.vx and p.vy == g.vy and p.vz == g.vz: 
                    impactedBody = 'g'
                    gImpacts += 1
                    impactTracker = gImpacts
                    planetVelocityComp = [p.vx, p.vy, p.vz]
                elif p.x == h.x and p.y == h.y and p.z == h.z and p.vx == h.vx and p.vy == h.vy and p.vz == h.vz: 
                    impactedBody = "h"
                    hImpacts += 1  
                    impactTracker = hImpacts
                    planetVelocityComp = [p.vx, p.vy, p.vz]
                else: # extract the impact velocity of the plantesimal 
                    planetesimalVelocityComp = [p.vx, p.vy, p.vz]
        
        #print(collided)
        
        relativeVelocityComp = [planetVelocityComp[0]-planetesimalVelocityComp[0],
                                planetVelocityComp[1]-planetesimalVelocityComp[1],
                                planetVelocityComp[2]-planetesimalVelocityComp[2]]
        
        relativeVelocity = sqrt((relativeVelocityComp[0])**2 + 
                                (relativeVelocityComp[1])**2 + 
                                (relativeVelocityComp[2])**2)
        
        # Simulation units are Mass in MSun, Distance in AU, and time in Year*2pi. 
        # Convert outputs to relevant units: 
        relativeVelocity = relativeVelocity*(2.0*np.pi)*(1/365)*(1/24)*(1/60)*(1/60)*(1.496e8) 
        collisionTime = sim.t/(2.0*np.pi)
        
        #print("Planetesimal collided with planet {} at time {} years with velocity {} km/s. Total collisions = {}".format( 
                #impactedBody, collisionTime, relativeVelocity, impactTracker))
        
        impactedBodies.append(impactedBody); impactTimes.append(collisionTime); 
        impactVelocity.append(relativeVelocity); totalImpacts.append(impactTracker)
    
        ### 
        
        cp1 = sim.particles[collided[0]] # points to first of the two collided particles
        cp2 = sim.particles[collided[1]] # points to second of the two collided particles
        
        # Merge by conserving mass, volume, and momentum
        
        invMass = 1.0/(cp1.m+cp2.m) # inverse sum mass of two collided particles
    
        # Conserve momentum
        cp1.vx = (cp1.vx*cp1.m + cp2.vx*cp2.m)*invMass
        cp1.vy = (cp1.vy*cp1.m + cp2.vy*cp2.m)*invMass
        cp1.vz = (cp1.vz*cp1.m + cp2.vz*cp2.m)*invMass 
        
        cp1.x = (cp1.x*cp1.m + cp2.x*cp2.m)*invMass 
        cp1.y = (cp1.y*cp1.m + cp2.y*cp2.m)*invMass 
        cp1.z = (cp1.z*cp1.m + cp2.z*cp2.m)*invMass
        
        # Conserve mass
        cp1.m = cp1.m + cp2.m
        
        # Conserve volume
        #cp1.r = (cp1.r*cp1.r*cp1.r + cp2.r*cp2.r*cp2.r)**(1.0/3.0)
        cp1.r = np.cbrt(cp1.r*cp1.r*cp1.r + cp2.r*cp2.r*cp2.r)
        
        # Remove particle with higher index (this will be the planetesimal, therefore keeping planet hash values)
        sim.remove(index = collided[1])
        
collisiondf = pd.DataFrame({'Impacted Body':impactedBodies, 'Collision Time (year)':impactTimes, 
                           'Impact Velocity (km/s)':impactVelocity, 'Total Impacts on Body':totalImpacts})

print(collisiondf)

rCust = h.r
mCust = h.m
xCust=h.x
yCust=h.y
zCust=h.z
vxCust = h.vx
vyCust = h.vy
vzCust = h.vz
eCust = h.e
iCust = h.inc
print(sim.status())
comCust = sim.calculate_com()

  Impacted Body  Collision Time (year)  Impact Velocity (km/s)  \
0             h               2.683418                7.269185   

   Total Impacts on Body  
0                      1  
---------------------------------
REBOUND version:     	3.17.0
REBOUND built on:    	May  6 2021 14:38:50
Number of particles: 	12
Selected integrator: 	mercurius
Simulation time:     	1.6860415063895353e+01
Current timestep:    	0.001300
---------------------------------
<rebound.particle.Particle object at 0x12154cec0, m=0.0898 x=-9.36351152343433e-07 y=-2.147356693732362e-06 z=-8.858633407927266e-09 vx=0.0001607878234691105 vy=9.711762376996173e-05 vz=4.6245514843369373e-07>
<rebound.particle.Particle object at 0x12157b7c0, m=4.1220000000000005e-06 x=-0.0050675045808504385 y=0.01032930327563014 z=4.747224842380202e-05 vx=-2.509194097771507 vy=-1.2364300101652588 vz=-0.006382892402831585>
<rebound.particle.Particle object at 0x12154cec0, m=3.924e-06 x=-0.012554286344502102 y=-0.009630835642532674 z=-

In [190]:
print("rBuilt =", rBuilt,"// rCust =", rCust)
print("mBuilt =", mBuilt,"// mCust =", mCust)
print("\n")
print("vxBuilt =", vxBuilt,"// vxCust =", vxCust)
print("vyBuilt =", vyBuilt,"// vyCust =", vyCust)
print("vzBuilt =", vzBuilt,"// vzCust =", vzCust)
print("\n")
print("xBuilt =", xBuilt,"// xCust =", xCust)
print("yBuilt =", yBuilt,"// yCust =", yCust)
print("zBuilt =", zBuilt,"// zCust =", zCust)

rBuilt = 3.208782809565695e-05 // rCust = 3.208782809565695e-05
mBuilt = 9.7803e-07 // mCust = 9.7803e-07


vxBuilt = 0.35960225324918227 // vxCust = 0.3596022532491809
vyBuilt = 1.156655944703193 // vyCust = 1.1566559447031943
vzBuilt = 0.003922235353419981 // vzCust = 0.0039222353534199925


xBuilt = 0.0587619999567121 // xCust = 0.058761999956712094
yBuilt = -0.018399055958415084 // yCust = -0.018399055958415084
zBuilt = -6.534364349358612e-05 // zCust = -6.534364349358611e-05


In [191]:
print(eBuilt, eCust)

0.006190729019866782 0.006190729019868561


In [200]:
print("CENTER OF MASS OF SIMULATION (BUILT-IN RESOLUTION):", "\n", comBuilt, "\n", 
      "CENTER OF MASS OF SIMULATION (CUSTOM RESOLUTION):", "\n", comCust)

CENTER OF MASS OF SIMULATION (BUILT-IN RESOLUTION): 
 <rebound.particle.Particle object at 0x12157b940, m=0.08981934415000001 x=5.167465286436949e-18 y=-1.8226437504804874e-17 z=-4.75488569279929e-21 vx=2.1965763644085877e-19 vy=-1.3264747132541386e-20 vz=-3.475568554040081e-21> 
 CENTER OF MASS OF SIMULATION (CUSTOM RESOLUTION): 
 <rebound.particle.Particle object at 0x12157b7c0, m=0.08981934415000001 x=5.167391609136013e-18 y=-1.8225700754279357e-17 z=-4.754741796212274e-21 vx=2.575635589012274e-19 vy=-1.6533034339274002e-20 vz=-3.5664296931075355e-21>


# This is the same code as the custom resolution block above, except that I have modified it to print out the lastcollision times after the first collision to highlight that it is not exactly the same as the simulation time when the collision error is raised. 

In [193]:
%%time

# import required packages
import rebound
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from rebound import hash as h
import pandas as pd

%matplotlib inline

sim = setupSimulation()

endTime = 500*2*np.pi

starImpacts = 0; bImpacts = 0; cImpacts = 0; dImpacts = 0; eImpacts = 0; fImpacts = 0; gImpacts = 0; hImpacts = 0

star = (sim.particles[h("star")]); b = (sim.particles[h("b")]); c = (sim.particles[h("c")]); d = (sim.particles[h("d")]); 
e = (sim.particles[h("e")]); f = (sim.particles[h("f")]); g = (sim.particles[h("g")]); h = (sim.particles[h("h")])

# initialize columns for dataframe
impactedBodies = []; impactTimes = []; impactVelocity = []; totalImpacts = [] 

while (sim.t != endTime): # while you still haven't reached the end of the intended simulation time... 
    
    try: # try to integrate to end of simulation time... 
        
        sim.integrate(endTime)
    
    except: # except if you run into an error (collision), run the following code: 
        
        collided = [] # initialize array that will store indices of collided particles 
        
        for particle in sim.particles: # loop through all particles in the simulation...  
            
            print("time =", sim.t, "lastcollision =", particle.lastcollision)
            if particle.lastcollision == sim.t:
                
                collided.append(particle.index)
           
        p = particle
                
        # Hardcoding to find the planet involved in collision by comparing particle parameters with 
        # planet parameters (retrieved by the hashes we declared in simulation set-up)
        if p.x == star.x and p.y == star.y and p.z == star.z and \
            p.vx == star.vx and p.vy == star.vy and p.vz == star.vz:                                                 
            impactedBody = 'star'
            starImpacts += 1  
            impactTracker = starImpacts
            planetVelocityComp = [p.vx, p.vy, p.vz]
        elif p.x == b.x and p.y == b.y and p.z == b.z and p.vx == b.vx and p.vy == b.vy and p.vz == b.vz: 
            impactedBody = 'b'
            bImpacts += 1
            impactTracker = bImpacts
            planetVelocityComp = [p.vx, p.vy, p.vz]
        elif p.x == c.x and p.y == c.y and p.z == c.z and p.vx == c.vx and p.vy == c.vy and p.vz == c.vz: 
            impactedBody = 'c'
            cImpacts += 1
            impactTracker = cImpacts
            planetVelocityComp = [p.vx, p.vy, p.vz]
        elif p.x == d.x and p.y == d.y and p.z == d.z and p.vx == d.vx and p.vy == d.vy and p.vz == d.vz: 
            impactedBody = 'd'
            dImpacts += 1
            impactTracker = dImpacts
            planetVelocityComp = [p.vx, p.vy, p.vz]
        elif p.x == e.x and p.y == e.y and p.z == e.z and p.vx == e.vx and p.vy == e.vy and p.vz == e.vz: 
            impactedBody = 'e'
            eImpacts += 1
            impactTracker = eImpacts
            planetVelocityComp = [p.vx, p.vy, p.vz]
        elif p.x == f.x and p.y == f.y and p.z == f.z and p.vx == f.vx and p.vy == f.vy and p.vz == f.vz: 
            impactedBody = 'f'
            fImpacts += 1
            impactTracker = fImpacts
            planetVelocityComp = [p.vx, p.vy, p.vz]
        elif p.x == g.x and p.y == g.y and p.z == g.z and p.vx == g.vx and p.vy == g.vy and p.vz == g.vz: 
            impactedBody = 'g'
            gImpacts += 1
            impactTracker = gImpacts
            planetVelocityComp = [p.vx, p.vy, p.vz]
        elif p.x == h.x and p.y == h.y and p.z == h.z and p.vx == h.vx and p.vy == h.vy and p.vz == h.vz: 
            impactedBody = "h"
            hImpacts += 1  
            impactTracker = hImpacts
            planetVelocityComp = [p.vx, p.vy, p.vz]
        else: # extract the impact velocity of the plantesimal 
            planetesimalVelocityComp = [p.vx, p.vy, p.vz]
        
        #print(collided)
        
        relativeVelocityComp = [planetVelocityComp[0]-planetesimalVelocityComp[0],
                                planetVelocityComp[1]-planetesimalVelocityComp[1],
                                planetVelocityComp[2]-planetesimalVelocityComp[2]]
        
        relativeVelocity = sqrt((relativeVelocityComp[0])**2 + 
                                (relativeVelocityComp[1])**2 + 
                                (relativeVelocityComp[2])**2)
        
        # Simulation units are Mass in MSun, Distance in AU, and time in Year*2pi. 
        # Convert outputs to relevant units: 
        relativeVelocity = relativeVelocity*(2.0*np.pi)*(1/365)*(1/24)*(1/60)*(1/60)*(1.496e8) 
        collisionTime = sim.t/(2.0*np.pi)
        
        #print("Planetesimal collided with planet {} at time {} years with velocity {} km/s. Total collisions = {}".format( 
                #impactedBody, collisionTime, relativeVelocity, impactTracker))
        
        impactedBodies.append(impactedBody); impactTimes.append(collisionTime); 
        impactVelocity.append(relativeVelocity); totalImpacts.append(impactTracker)
    
        ### 
        
        cp1 = sim.particles[collided[0]] # points to first of the two collided particles
        cp2 = sim.particles[collided[1]] # points to second of the two collided particles
        
        # Merge by conserving mass, volume, and momentum
        
        # Conserve mass
        
        invMass = 1.0/(cp1.m+cp2.m) # inverse sum mass of two collided particles
    
        # Conserve momentum
        cp1.vx = (cp1.vx*cp1.m + cp2.vx*cp2.m)*invMass
        cp1.vy = (cp1.vy*cp1.m + cp2.vy*cp2.m)*invMass
        cp1.vz = (cp1.vz*cp1.m + cp2.vz*cp2.m)*invMass 
        
        cp1.x = (cp1.x*cp1.m + cp2.x*cp2.m)*invMass 
        cp1.y = (cp1.y*cp1.m + cp2.y*cp2.m)*invMass 
        cp1.z = (cp1.z*cp1.m + cp2.z*cp2.m)*invMass
        
        cp1.m = cp1.m + cp2.m
        
        # Conserve volume
        #cp1.r = (cp1.r*cp1.r*cp1.r + cp2.r*cp2.r*cp2.r)**(1.0/3.0)
        cp1.r = np.cbrt(cp1.r*cp1.r*cp1.r + cp2.r*cp2.r*cp2.r)
        
        # Remove particle with higher index (this will be the planetesimal, therefore keeping planet hash values)
        sim.remove(index = collided[1])
        
collisiondf = pd.DataFrame({'Impacted Body':impactedBodies, 'Collision Time (year)':impactTimes, 
                           'Impact Velocity (km/s)':impactVelocity, 'Total Impacts on Body':totalImpacts})

#print(collisiondf)
sim.status()

time = 16.86127284098699 lastcollision = 0.0
time = 16.86127284098699 lastcollision = 0.0
time = 16.86127284098699 lastcollision = 0.0
time = 16.86127284098699 lastcollision = 0.0
time = 16.86127284098699 lastcollision = 0.0
time = 16.86127284098699 lastcollision = 0.0
time = 16.86127284098699 lastcollision = 0.0
time = 16.86127284098699 lastcollision = 16.860515266972953
time = 16.86127284098699 lastcollision = 0.0
time = 16.86127284098699 lastcollision = 0.0
time = 16.86127284098699 lastcollision = 0.0
time = 16.86127284098699 lastcollision = 0.0
time = 16.86127284098699 lastcollision = 16.860515266972953


IndexError: list index out of range